In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing as housing
from sklearn.model_selection import train_test_split

In [3]:
K.clear_session()
np.random.seed(55)
tf.random.set_seed(55)

In [4]:
X = housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(X.data, X.target.reshape(-1, 1))
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full.reshape(-1, 1))

X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

X_train_scaled = (X_train - X_mean)/X_std
X_valid_scaled = (X_valid - X_mean)/X_std
X_test_scaled = (X_test - X_mean)/X_std

In [5]:
assert X_train_scaled.shape == X_train.shape
assert X_valid_scaled.shape == X_valid.shape
assert X_test_scaled.shape == X_test.shape

In [6]:
@tf.function
def my_relu(x):
    return tf.math.maximum(0., x)

@tf.function
def my_elu(x):
    return tf.where(x>=0, x, tf.exp(x)-1)

In [7]:
class GlorotNormal(tf.keras.initializers.Initializer):
    def __init__(self, mean=0., stddev=1.):
        self.mean = mean
        self.stddev = stddev
    def __call__(self, shape, dtype=None):
        return tf.random.normal(
            shape, mean=self.mean, stddev=self.stddev, dtype=dtype)
    def get_config(self):
        return {"mean": self.mean, "stddev": self.stddev}

In [8]:
class ZerosInitializer(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        return tf.zeros(shape=shape, dtype=dtype)

In [9]:
class custom_Dense(keras.layers.Layer):
    def __init__(self, units=1, 
                 activation=None, 
                 use_bias=True, 
                 **kwargs):
        super().__init__(**kwargs)
        self.units = units
        if activation:
            if activation=="relu":
                self.activation = my_relu
            elif activation=="elu":
                self.activation = my_elu
        else:
            self.activation=(lambda x: x)
        self.use_bias=use_bias
        
    def build(self, input_shape):
        print("hello", type(input_shape[0]), type(input_shape[1]), type(input_shape))
        shape = (input_shape[-1], self.units)
        stddev = tf.sqrt(2. / (input_shape[-1]+self.units))
        weight_init = GlorotNormal(stddev=stddev)
        self.kernel = tf.Variable(
            initial_value=weight_init(shape=shape, 
                                      dtype='float32'), trainable=self.trainable)
        if self.use_bias:
            bias_init = ZerosInitializer()
            self.bias = tf.Variable(
                initial_value=bias_init(shape=(self.units), 
                                        dtype='float32'), trainable=self.trainable)
        else:
            self.bias = None
        self.built = True
#         super().build(input_shape)

    def call(self, inputs):
        return self.activation(inputs @ self.kernel + self.bias)

    def compute_output_shape(self, input_shape):
        return [input_shape[0], self.units]

    def get_config(self):
        super_config = super().get_config()
        return {**super_config, 
                "units": self.units, 
                "activation": activation}

In [10]:
class custom_nDense(keras.layers.Layer):
    def __init__(self, 
                 units=1, 
                 n_layers=1, 
                 activation=None, 
                 use_bias=True, 
                 **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.n_layers=n_layers
        self.activation=activation
        self.hidden= [custom_Dense(units=units, 
                                  activation=activation, 
                                  use_bias=use_bias) 
                      for _ in range(self.n_layers)]
        
    def call(self, X):
        Z = X
        for layer in self.hidden:
            Z = layer(Z)
        return Z
    
    def compute_output_shape(self, input_shape):
        return [input_shape[0], self.units]
    
    def get_config(self):
        super_config = super().get_config()
        return {**super_config, 
                "units":self.units, 
                "activation":self.activation}

In [11]:
class MyModel(keras.Model):
    def __init__(self, 
                 output_dim=1, 
                 **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = custom_nDense(units=100, n_layers=1, activation="relu")
        self.out = custom_Dense(units=output_dim)
        
    def call(self, X):
        Z = self.hidden1(X)
        return self.out(Z)

In [12]:
dir(MyModel)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_attribute_sentinel',
 '_base_init',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_args',
 '_call_full_argspec',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_collect_input_masks',
 '_compile_was_called',
 '_compute_dtype',
 '_compute_tensor_usage_count',
 '_conform_to_reference_input',
 '_dedup_weights',
 '_deferred_dependencies',
 '_dtype',
 '_eager_add_metric',
 '_

In [13]:
K.clear_session()
np.random.seed(55)
tf.random.set_seed(55)

In [14]:
model = MyModel(1)

In [15]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-4), metrics=["mae"])

In [16]:
model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid), batch_size=10)

Epoch 1/10
hello <class 'int'> <class 'int'> <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
hello <class 'int'> <class 'int'> <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
1161/1161 [==============================] - 2s 2ms/step - loss: 3.5285 - mae: 1.5200 - val_loss: 1.9845 - val_mae: 1.0458
Epoch 2/10
1161/1161 [==============================] - 2s 2ms/step - loss: 1.4947 - mae: 0.8828 - val_loss: 1.1116 - val_mae: 0.7433
Epoch 3/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.9800 - mae: 0.7175 - val_loss: 0.8244 - val_mae: 0.6560
Epoch 4/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.8075 - mae: 0.6608 - val_loss: 0.7112 - val_mae: 0.6213
Epoch 5/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.7383 - mae: 0.6338 - val_loss: 0.6616 - val_mae: 0.6040
Epoch 6/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.7051 - mae: 0.6188 - val_loss: 0.6352 - val_mae: 0.5932
Ep

In [17]:
model.layers

In [18]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custom_n_dense (custom_nDens multiple                  900       
_________________________________________________________________
custom__dense_1 (custom_Dens multiple                  101       
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [19]:
X_train_scaled

array([[-1.01105039e+00, -6.98123209e-01, -1.94180473e-01, ...,
         2.92666074e-01,  4.22302244e-01, -4.57247389e-01],
       [ 7.46565256e-03, -6.98123209e-01,  1.67477270e-01, ...,
         4.81086670e-02,  1.30748079e+00, -1.56103408e+00],
       [ 6.62417881e-01, -9.37182054e-01,  2.34491761e-01, ...,
         1.62800374e-03, -8.46922025e-01,  8.55363807e-01],
       ...,
       [ 2.30366463e-01,  1.85183780e+00,  3.67541077e-01, ...,
        -9.94059513e-02, -7.20467947e-01,  7.16147468e-01],
       [ 1.10895706e+00, -9.37182054e-01,  1.81766465e-01, ...,
        -9.77141916e-03,  7.59513119e-01, -1.19310518e+00],
       [ 6.04993659e-01, -9.37182054e-01,  2.81134684e-01, ...,
        -1.29788671e-02,  5.48756322e-01, -1.02902878e+00]])

In [20]:
K.clear_session()
np.random.seed(55)
tf.random.set_seed(55)

In [21]:
model = keras.models.Sequential()
initializer=keras.initializers.GlorotNormal()
model.add(keras.layers.Dense(100, activation="relu", kernel_initializer=initializer))
model.add(keras.layers.Dense(1))

In [22]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-4), metrics=["mae"])

In [23]:
model.layers

In [24]:
model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid), batch_size=10)

Epoch 1/10
1161/1161 [==============================] - 2s 2ms/step - loss: 2.9706 - mae: 1.3388 - val_loss: 2.0143 - val_mae: 1.0008
Epoch 2/10
1161/1161 [==============================] - 2s 2ms/step - loss: 1.4925 - mae: 0.8831 - val_loss: 1.2659 - val_mae: 0.7735
Epoch 3/10
1161/1161 [==============================] - 2s 2ms/step - loss: 1.0426 - mae: 0.7443 - val_loss: 0.9347 - val_mae: 0.6881
Epoch 4/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.8622 - mae: 0.6836 - val_loss: 0.7821 - val_mae: 0.6463
Epoch 5/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.7802 - mae: 0.6504 - val_loss: 0.7099 - val_mae: 0.6234
Epoch 6/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.7377 - mae: 0.6313 - val_loss: 0.6698 - val_mae: 0.6087
Epoch 7/10
1161/1161 [==============================] - 2s 2ms/step - loss: 0.7114 - mae: 0.6187 - val_loss: 0.6463 - val_mae: 0.5980
Epoch 8/10
1161/1161 [==============================] - 2s 2ms

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  900       
_________________________________________________________________
dense_1 (Dense)              multiple                  101       
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [26]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [27]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [28]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [29]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

In [30]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)

In [31]:
import time

mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000


In [32]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [34]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
#         if step == n_steps:
#             print(" ", type(main_loss), main_loss, type(model.losses), model.losses, sep="\n")
        gradients = tape.gradient(loss, model.trainable_variables)
#         if step == n_steps and epoch == n_epochs:
#             print("", type(gradients), len(gradients), gradients, sep="\n")
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

11610/11610 [==============================] - mean: 1.2608 - mean_absolute_error: 0.5807
Epoch 2/5
11610/11610 [==============================] - mean: 0.6838 - mean_absolute_error: 0.5412
Epoch 3/5
11610/11610 [==============================] - mean: 0.6820 - mean_absolute_error: 0.5465
Epoch 4/5
11610/11610 [==============================] - mean: 0.6599 - mean_absolute_error: 0.5399
Epoch 5/5
11610/11610 [==============================] - mean: 0.6698 - mean_absolute_error: 0.5424


In [35]:
model.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.07555242>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.060151465>]